## Exercise Data Quality Analytical Exercise
##### By: Abodunde Ojo

#### Import Necessary Libraries

In [1]:
import pandas as pd
import numpy as np

# Libraries to help with data visualization
import matplotlib.pyplot as plt
import seaborn as sns

#import library for Outlier Detection
from scipy import stats

# Date Library
from dateutil.parser import parse
import datetime
today = datetime.date.today()
from datetime import datetime

# to suppress warnings
import warnings
warnings.filterwarnings("ignore")

#### Import Data

In [2]:
## Complete the code to import the data
df = pd.read_excel("Data.xlsx", sheet_name="AccountDataDQA")

In [3]:
# copying data to another variable to avoid any changes to original data
data = df.copy()

#### Remove Spaces from Column Name

In [5]:
columns = [column.strip()
          for column in data.columns]

In [6]:
#Replace Existing Columns with clean ones
data.columns = columns

In [7]:
#Check Shape
data.shape
# This shows it in the format (rows, columns)

(27648, 11)

### OBSERVATION
I passed a length check on the **acc_status column** and discovered that there are spaces and it is giving a length of 10 instead of 1. This is shown in the next slide.

In [8]:
lenghth_of_acc_status = [len(x) for x in data['acc_status'][0:20]]
lenghth_of_acc_status

[10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10]

#### Treat the 'acc_status' and 'account_type' columns by removing the spaces (Ideally, this should be done on all column)

In [9]:
data['acc_status'] = data['acc_status'].str.strip()
data['account_type'] = data['account_type'].str.strip()

##### Copy the DataFrame so that we can use it for spooling clean data Later

In [10]:
cleaned_df = data.copy()

## Check Data Type Conformance

#### Import (and Clean) the Data Dictionary from the sheet and compare Data types

In [11]:
meta_data = pd.read_excel(r"C:\Users\ojoabo\Downloads\DQArecruitmenttestV2.xlsx", sheet_name="Data Quality Interview Assessme")

In [12]:
#View the Data Dictionary
meta_data

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,NaN,Field Reference,Description,Character Type,Maximum Length
1,NaN,acc_id,Anonymised account ID,NUMBER,22
2,NaN,account_type,2 digit code to show the type of account,CHARACTER,2
3,NaN,credit_turnover,"Credit turnover, in GBP, the total value of a...",NUMBER,22
4,NaN,debit_turnover,"Debit turnover, in GBP, the total value of al...",NUMBER,22
5,NaN,month_data_date,Date associated with the data provided for a g...,CHARACTER,8
6,NaN,acc_status,"Status code, monthly indication of the payment...",CHARACTER,1
7,NaN,min_balance,"Minimum balance during the period, in GBP, a n...",NUMBER,22
8,NaN,balance,"Total outstanding balance in GBP, a negative v...",NUMBER,22
9,NaN,rejected_payments,"Rejected payments, the number of cheques, dire...",NUMBER,22


In [13]:
# Clean the Dataframe
meta_data = meta_data.drop('Unnamed: 0', axis ='columns')

In [14]:
# Select the first row as the column and drop it from the data dictionary
meta_data.columns = meta_data.loc[0]
meta_data = meta_data.drop(0, axis =0)

In [15]:
#View the data dictionary
meta_data

,Field Reference,Description,Character Type,Maximum Length
1,acc_id,Anonymised account ID,NUMBER,22
2,account_type,2 digit code to show the type of account,CHARACTER,2
3,credit_turnover,"Credit turnover, in GBP, the total value of a...",NUMBER,22
4,debit_turnover,"Debit turnover, in GBP, the total value of al...",NUMBER,22
5,month_data_date,Date associated with the data provided for a g...,CHARACTER,8
6,acc_status,"Status code, monthly indication of the payment...",CHARACTER,1
7,min_balance,"Minimum balance during the period, in GBP, a n...",NUMBER,22
8,balance,"Total outstanding balance in GBP, a negative v...",NUMBER,22
9,rejected_payments,"Rejected payments, the number of cheques, dire...",NUMBER,22
10,monthly_payment_scheduled,"Payment amount, the normal monthly payment (in...",NUMBER,22


**Note**: Before diving into the analysis and analyzing data quality metrics, we first check if the **character type** is the same as the specification provided in the Data Dictionary

In [16]:
# Get data types from the data frame
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27648 entries, 0 to 27647
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   acc_id                     27648 non-null  int64  
 1   account_type               27624 non-null  object 
 2   credit_turnover            27642 non-null  float64
 3   debit_turnover             27642 non-null  float64
 4   month_data_date            27648 non-null  object 
 5   acc_status                 27648 non-null  object 
 6   min_balance                27642 non-null  object 
 7   balance                    27642 non-null  object 
 8   rejected_payments          27648 non-null  object 
 9   monthly_payment_scheduled  27648 non-null  object 
 10  credit_limit               27648 non-null  object 
dtypes: float64(2), int64(1), object(8)
memory usage: 2.3+ MB


#### Get all the Fields that should be "Numerical"

In [17]:
should_be_numerical_fields =[]
for index, row in meta_data.iterrows():
    if row['Character Type'] == 'NUMBER':
        should_be_numerical_fields.append(row['Field Reference'])
        
should_be_numerical_fields = [field.strip() for field in should_be_numerical_fields]

#### Get all the fields that should be "Character"

In [18]:
should_be_character_fields= []
for index, row in meta_data.iterrows():
    if row['Character Type'] == 'CHARACTER':
        should_be_character_fields.append(row['Field Reference'])
        
should_be_character_fields = [field.strip() for field in should_be_character_fields]

#### Get All the fields that are (in fact) Numerical

In [19]:
numeric_columns = data.select_dtypes(include=np.number).columns.tolist()

#### Get all Non-conforming Fields

In [20]:
faulty_fields = []
for column in should_be_numerical_fields:
    if column not in numeric_columns:
        faulty_fields.append(column)

In [21]:
print(faulty_fields)

['min_balance', 'balance', 'rejected_payments', 'monthly_payment_scheduled', 'credit_limit']


**Observation**: As a first observation we can see that there is a mismatch of the Data types in the Data Frame and the Provided Data Dictionary.

***1.*** min_balance, balance, rejected_payments, monthly_payment_scheduled, and credit_limit should be numerical variables but are captured as text (objects) in this dataframe.

***2.*** Date Should be captured as date.

***3.*** acc_id, acc_type, and acc_status are appropriately captured as text (categorical) data **| This would be confirmed**

***4.*** credit_turnover and debit_turnover are appropriately captured as numerical data

### Create a DataFrame that captures all rows with errors
This dataframe attaches the rows and the cause of the exception (Data Quality Issue). It contains the normal rows and an additional row that captures the cause of exception

In [22]:
Errorsss = {'acc_id': [], 'account_type': [], 'credit_turnover': [], 'debit_turnover': [],
       'month_data_date': [], 'acc_status': [], 'min_balance': [], 'balance': [],
       'rejected_payments': [], 'monthly_payment_scheduled': [], 'credit_limit': [], 'error_code': []}

Errorsss_df = pd.DataFrame(Errorsss)
Errorsss_df.columns

Index(['acc_id', 'account_type', 'credit_turnover', 'debit_turnover',
       'month_data_date', 'acc_status', 'min_balance', 'balance',
       'rejected_payments', 'monthly_payment_scheduled', 'credit_limit',
       'error_code'],
      dtype='object')

##### Define a function for capturing the error

In [23]:
def append_error(value, error, error_dataframe):
    new_row = {'acc_id':value['acc_id'], 'account_type':value['account_type'], 'credit_turnover':value['credit_turnover'], 
                   'debit_turnover':value['debit_turnover'], 'month_data_date':value['month_data_date'], 'acc_status':value['acc_status'], 
                   'min_balance':value['min_balance'], 'balance':value['balance'],'rejected_payments':value['rejected_payments'], 
                   'monthly_payment_scheduled':value['monthly_payment_scheduled'], 
                   'credit_limit':value['credit_limit'], 'error_code':error}
    error_dataframe = error_dataframe.append(new_row, ignore_index=True)
    return error_dataframe


**Function Explanation**: For every columns that we do a quality check on, this function appends the wrong entry and the reason why there is an error. The final table will be exported and can be sent to the business owners to work on the errors.

### Quick Modification

#### Change the date column to YYYYMMDD format as specified in the data dictionary

In [24]:
# Convert month_data_date to YYYYMMDD format
#data['month_data_date'] = data['month_data_date'].dt.strftime('%Y%m%d')

#### Define Row by row date conversion function

In [25]:
def date_converstion (x):
    try:
        x_string = datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S')
        x_string = x_string.strftime('%Y%m%d')
        return x_string
    except Exception as e:
        return False

assert date_converstion('123456') == False

##### Define Generic Function for Finding Non-compliant (Non-Numeric) rows in Numeric Columns

In [26]:
def is_digit(x):
    if len(x) <= 22:
        try:
            int(x)
            return True
        except ValueError:
            return  False
    else:
        return False

What this function does, is that it returns **"True"** if a value can be converted to number. If it cannot be converted then it is a non-numerical entry and would return **"False"**.

###### Define generic function to measure uniqueness  (to be used where applicable)


In [27]:
def count_unique (x):
    unique_value = data[x].nunique()
    return unique_value

## Preliminary Checks (Missing and Duplicated Data)
#### Missing (Incomplete) Values

In [28]:
# checking for missing values in the data
data.isna().sum()

acc_id                        0
account_type                 24
credit_turnover               6
debit_turnover                6
month_data_date               0
acc_status                    0
min_balance                   6
balance                       6
rejected_payments             0
monthly_payment_scheduled     0
credit_limit                  0
dtype: int64

**Observation**: As can be seen ***Account Type*** has **24** Missing Values, while ***Credit_turnover, Debit_turnover, Minimum_Balance, and Balance*** all have 6 missing Values

#### Total Values

In [29]:
for column in data.columns:
    print(f'For Column {column.strip()}: There are {data[column].value_counts().sum()} entries.')

For Column acc_id: There are 27648 entries.
For Column account_type: There are 27624 entries.
For Column credit_turnover: There are 27642 entries.
For Column debit_turnover: There are 27642 entries.
For Column month_data_date: There are 27648 entries.
For Column acc_status: There are 27648 entries.
For Column min_balance: There are 27642 entries.
For Column balance: There are 27642 entries.
For Column rejected_payments: There are 27648 entries.
For Column monthly_payment_scheduled: There are 27648 entries.
For Column credit_limit: There are 27648 entries.


#### Duplicated Rows

In [30]:
# checking for duplicated rows in the data
data.duplicated().sum()

0

**Observation**: There are no duplicated entries in the dataset.

In [31]:
# Account Id is expected to be unique. We can check this.
repeated_accounts = data.shape[0] - data['acc_id'].nunique()
print( f'Although anonymised, there are {repeated_accounts} repeated account ids')

Although anonymised, there are 23043 repeated account ids


#### Unique Values

In [32]:
for column in data.columns:
    print(f'There are {data[column].nunique()}. unique values in the {column} column.')

There are 4605. unique values in the acc_id column.
There are 3. unique values in the account_type column.
There are 5931. unique values in the credit_turnover column.
There are 6859. unique values in the debit_turnover column.
There are 24. unique values in the month_data_date column.
There are 1. unique values in the acc_status column.
There are 14058. unique values in the min_balance column.
There are 14622. unique values in the balance column.
There are 12. unique values in the rejected_payments column.
There are 14. unique values in the monthly_payment_scheduled column.
There are 6. unique values in the credit_limit column.


### Defining Functions: Rules of the Data
**These functions are being defined mostly to let us loop through the columns later and use them in the data quality metrices that would be calculated.**

For this exercise, the data quality measure would be benchmarked against the specifications in the Data Dictionary provided with the Data. This means that we would check the conformance each column against the rules in the data dictionary. For some, there are general rules such as the lenght of the characters. For these columns, a generic function can be passed to check conformance. We will start with generic rules

#### Maximum Length Function
This shows the maximum length of characters allowed for each field. Here we map it together

In [33]:
#First we get the maximum lengths of the fields from the data dictionary
fields = [field.strip() for field in meta_data['Field Reference']] #code to get the fields from the data dictionary
counttt = [lenght for lenght in meta_data['Maximum Length']] #Code to get the maximum length from the data dictionary
field_lengths = dict(zip(fields,counttt)) #Code to map each field to maximum length in a dictionaryfor enable us call it later
field_lengths

{'acc_id': 22,
 'account_type': 2,
 'credit_turnover': 22,
 'debit_turnover': 22,
 'month_data_date': 8,
 'acc_status': 1,
 'min_balance': 22,
 'balance': 22,
 'rejected_payments': 22,
 'monthly_payment_scheduled': 22,
 'credit_limit': 22}

In [34]:
def numerical_character_and_lenght_check(x):
    global Errorsss_df #Declare this as Global to effect changes outside the function
    count = 0
    wrong_input =[]
    for index, value in data.iterrows():
        new_value = value[x]
        if len(str(new_value))<= field_lengths[x]:
            try:
                int(str(new_value))
                count +=1
            except ValueError:
                try: # This is here as the first 'try' might not always work for float object types
                    int(new_value)
                    count +=1
                except Exception as e:
                    Errorsss_df = append_error(value, f'input in {x} column is not a number', Errorsss_df)
                    wrong_input.append(new_value)
        else:
            Errorsss_df = append_error(value, f'input in {x} column does not meet length requirement', Errorsss_df)
            wrong_input.append(new_value)
    return count

In [35]:
numerical_character_and_lenght_check('rejected_payments')

27647

**Function Explanation:** This particular function is used to check the compliance of the Numerical columns. It first checks if the data entry conforms with the maximum length specified in the data dictionary. If it does, it then checks if it is a numerical value (That is, if all caharacters in the entry are numbers or can be converted to numbers). If these requirements are met, it adds 1 to the count value. If it is a string value that cannot be converted to a numerical value, it adds zero to the count value and appends the wrong input into a list. At the end, all wrong entries are printed and the total number of correct entries are returned.

In [36]:
def check_greater_than(x, y):
    if x > y:
        raise ValueError(f"{x} is greater than {y}")

**Function Explanation**: This will be used to check if there are anomalies in the date column. It is not expected that any date will be greater than today in the data that was provided

In [37]:
def text_character_and_length_check(x):
    global Errorsss_df
    count = 0
    wrong_input =[]
    #column_type = data[x].dtype
    if x == 'month_data_date':
        for index, value in data.iterrows():
            new_value = value[x]
            if date_converstion(new_value) != False and len(date_converstion(new_value)) == field_lengths[x]:
                try:
                    check_greater_than(new_value.date(),today)
                    parse(str(date_converstion(new_value)))
                    count += 1
                except Exception as e:
                    Errorsss_df = append_error(value, f'input in {x} column is an outlier date', Errorsss_df)
                    wrong_input.append(new_value)
            else:
                Errorsss_df = append_error(value, f'input in {x} column is not a valid date/exceeds length requirement', Errorsss_df)
                wrong_input.append(new_value)

    else:
        for index, value in data.iterrows():
            new_value = value[x]
            if len(str(new_value)) <= field_lengths[x]:
                count +=1
            else:
                Errorsss_df = append_error(value, f'input in {x} column exceeds length requirement', Errorsss_df)
                wrong_input.append(new_value)
    return count
    #print(wrong_input)
    #print(count)
        

In [38]:
text_character_and_length_check('month_data_date')

27636

In [39]:
should_be_character_fields

['account_type', 'month_data_date', 'acc_status']

**Function Explanation**: This function works on two types of data and it has two procedures. First the function checks if the column passed into it is the Date Column. If it is, the function iterates through each row of the column and checks if it can be converted into the specified YYYYMMDD format. If it can be converted, it then checks if the length meets the required length. If all these prior requirements have been met, we then check if the date is not an outlier (we check this by ensuring that the date is not higher than a set dat - in the case of this code, the present date. Finally, if these checks are passed, we try to parse it (meaning we try to check if the text can be converted to data) if it can, then it is a correct entry. Otherwise, it is a wrong entry.

For the second funtion, it works with the character fields and checks if the length requirement is met

In [40]:
assert date_converstion('1234') ==False

In [41]:
text_character_and_length_check('acc_status')

27648

In [42]:
should_be_character_fields

['account_type', 'month_data_date', 'acc_status']

In [43]:
def value_lenght_check(x):
    count = 0
    for value in data[x]:
        if len(str(value))<= field_lengths[x]:
            count +=0
        else:
            count +=0
    return count

**Function Explanation**: This function checks if each entry meets the specified data dictionary limit for that column. If it meets it, the count is incresed by one. If it does not, the count does not increase. The entry then returns the total number of entries that meet the specified length requirement. This is used for checking **completness of the columns**

In [44]:
def numerical_character_check(x):
    count = 0
    for value in data[x]:
        try:
            int(str(value))
            count +=1
        except ValueError:
            try: # This is here as the first 'try' might not work for float object types
                int(value)
                count +=1
            except Exception as e:
                count +=0
    else:
        count +=0
    return count
    #print(wrong_input)
    #print (count)

**Function Explanation**: This function checks if the the specified character rules are followed such as the data type entry. This particular function is only valid for numerical columns. This is useful for checking **consistency**

In [45]:
def text_character_check(x):
    count = 0
    if x == 'month_data_date':
        for value in data[x]:
            if date_converstion(value) != False:
                try:
                    parse(str(date_converstion(value)))
                    count += 1
                except Exception as e:
                    count += 0
            else:
                count += 0

    else:
        for value in data[x]:
            try:
                str(value).isalpha()
                count +=1
            except Exception as e:
                count +=0
    return count

**Function Explanation**: This function checks if the the specified description rules are followed such as the data type entry. This particular function is only valid for character (object) columns and the date column. It also mostly focuses on the data type. This is useful for checking **consistecy**

In [46]:
def null_values_check(x):
    global Errorsss_df
    count = 0
    for index, value in data.iterrows():
        new_value = value[x]
        if pd.isna(new_value):
            count +=1
            Errorsss_df = append_error(value, f'Input in column {x} is null', Errorsss_df)
        else:
            pass
        
    return count

**Function Explanation**: This function merely checks whethere there are missing values or not and returns the total number of missing values in the specified column. Likewise, null values are added to the null error dataframe. This measures **completeness** of the data

In [47]:
def valid_and_outlier_check(x):
    global Errorsss_df
    global data
    outlier_count =0
    outlier_values =[]
    valid_entry =[]
    valid_entry_2 =[]
    wrong_entry =[]
    count_of_valid_entry_non_outliers = data.shape[0]
    for index, value in data.iterrows():
        new_value = value[x]
        try: 
            int(str(new_value)) #Check if it can be converted to a number
            valid_entry.append(abs(int(new_value))) # if it can be converted to a number, attach the value to the "valid_entry" list
            valid_entry_2.append(int(new_value)) # attach it to valid_entry_2 list as well
            #count_of_valid_entry_non_outliers +=1
        except ValueError: #if it throws an error 
            try:
                int(new_value) #check if it a float
                valid_entry.append(int(new_value)) #if it is attach it to valid_entry
                valid_entry_2.append(int(new_value)) #Also attach it to valid_entry_2
                #count_of_valid_entry_non_outliers +=1
            except Exception as e: #If it is neither a float or number
                Errorsss_df = append_error(value, f'input in {x} column is not a number', Errorsss_df)
                valid_entry_2.append(np.nan) # Add null for all wrong entries or null values
                wrong_entry.append(new_value)
                count_of_valid_entry_non_outliers -=1
    mean_of_valid_entries = sum(valid_entry)/len(valid_entry)
    valid_entry_2 = [mean_of_valid_entries if np.isnan(x) else x for x in valid_entry_2]
    data['temp_values'] = valid_entry_2 #Create a new column with replaced faoulty rows
    
    
    
# When it is done creating the DataFrame, it would then check if there are outliers using stats library from scipy
    z_scores = np.abs(stats.zscore(data['temp_values']))
    data['z_scores'] = z_scores
    threshold = 3 #Usually, this can be calculated using inter quatile range of the data
    data['outliers'] = data['z_scores']>threshold
    
    for index, value in data.iterrows():
        new_value = value['outliers']
        if new_value == True:
            outlier_count +=1
            count_of_valid_entry_non_outliers -=1
            Errorsss_df = append_error(value, f'input in {x} column is an outlier', Errorsss_df)
        else:
            pass
    for index, row in data.iterrows():
        if row['outliers'] == True:
            outlier_values.append(row[x])
        else:
            outlier_count+=0

    added_columns = ['outliers', 'z_scores', 'temp_values']
    data = data.drop(added_columns, axis = 1)
    results = [outlier_count, outlier_values, wrong_entry, count_of_valid_entry_non_outliers]
    return results

**Function Explanation**: This function helps us check whether there are anomalies in data entries of numerical columns. Specifically, checks for outliers in any specified numerical column. However, there are instances where entries in a numerical column are not numeric (perhaps text?), the function removes them as well. At the end, it attaches the outliers, the wrong entries and the total count of outliers. This can help the data owners identify the wrong and outlier entries.

valid_and_outlier_check(x)[0] returns the count of outliers

valid_and_outlier_check(x)[1] returns the outlier values

valid_and_outlier_check(x)[2] returns the wrong entries

valid_and_outlier_check(x)[3] returns the total number of valid entries.

In [48]:
valid_and_outlier_check('monthly_payment_scheduled')

[3, [54545, 55550, 25000], ['A'], 27644]

## Data Quality (DQ) Metrices

Data Quality has a number of key dimensions which we would look at at during this stage. Some of the Dimentions that would be analysed include:

**Completeness**: This measures how the degree to which the data set contains all the necessary information and records required to meet the set objectives. This will be captured by **null values**

**Consistency**: This measures the degree to which there is uniformity in the datase. That is, it checks if the data format is the same in different sources. As we are not working with data from multiple sources, we would not capture this metric.

**Validity**: This is measured by the degree to which the dataset meets defined standards and rules. Adherence to rules set in the data dictionary will serve as a guide in measuring this. 

**Accuracy**: The degree to which the data reflects real world values and actual events. For applicable fields, this will be captured using outliers and wrong entries.

**Duplication & Uniqueness**: this is captured by the degree to which data contains duplicate records, which can lead to errors and inconsistencies. Duplication looks at it from row to row to check for duplicates while uniqueness looks at columns that are expected to be unique to find duplicates in such column.

**Other Dimensions**: These are other dimensions of data quality that are not captured in this analysis. They include: Timeliness, Relevance, Accessibility, Interpretability, Integrity, Consensus, Bias, Consolidation, Currency, Precision, etcetera.

###### Validity
Since the shape of the data is (27648, 11), we expect that there are a total of 304128 entires in the dataset. Validity aims to loop through every single column in the dataset and identify every single cell that conforms with the specified rules of the columns, and is not null. This means that we are benchmarking each cell against the data description of the columns, the null rule (no null values) and the character length rule. This is divided by the total number of expected entries and multipled by 100.

In [49]:
def validity ():
        total_valid_rows = 0
        for column in should_be_character_fields:
            total_valid_rows += text_character_and_length_check(column)
        for column in should_be_numerical_fields:
            total_valid_rows += numerical_character_and_lenght_check(column)
        return (total_valid_rows/(data.shape[1]*data.shape[0]))*100

###### Completeness
Completeness checks wethere there are any missing values in the data set. As stated earlier, there are 304128 expected entries. We measure completeness by dividing the total filled cells by the total expected entried and multiplying it by 100

In [50]:
def completeness():
    number_of_expected_entries = data.shape[1]*data.shape[0]
    total_null_rows = 0
    for column in data.columns:
        total_null_rows += null_values_check(column)
    return (((number_of_expected_entries - total_null_rows)/number_of_expected_entries)*100)

###### Uniqueness
Not every field is expected to have unique values but the account ID although anonymised is expected to be unique. As such Uniqueness will only be checked for the account id column. This is measured below and the count_unique function is called

In [51]:
def uniqueness():
    unique_entries = count_unique('acc_id')
    return (unique_entries/data.shape[0])*100

#### Duplication
We can check if there are duplicate entries in the dataset. For this, a simple python function is inbuilt for this. We would write a simple function that simplly calculates the duplication score.


In [52]:
def duplication():
    number_of_duplicates = data.duplicated().sum()
    return (((data.shape[0]-number_of_duplicates)/data.shape[0])*100)

###### Accuracy
For accuracy, we will check whether the inputs (especially) in the numerical columns represent real life scenarios. For example, we can check for outliers in numerical columns that do not make sense. Likewise, we can check if the monthly payment is greater than the outstanding amount. In such a scenario, we can ascertain that the entry is invalid. 

In [53]:
#I will just manually specify the columns I want to run accuracy on but it can be automated to include all numeric fields
accuracy_columns= ['credit_turnover', 'debit_turnover', 'min_balance', 'balance',  
                   'rejected_payments', 'monthly_payment_scheduled', 'credit_limit'] 
def accuracy():
    count_accurate = 0
    total_applicable_rows = (data.shape[0]*len(accuracy_columns)) # to Get total rows considered
    for column in accuracy_columns:
        count_accurate += valid_and_outlier_check(column)[3]
    return ((count_accurate/total_applicable_rows)*100)
    #result = [count_accurate, total_applicable_rows]
    #return result

### Final DQ Score
We compute two score:

1. DQ Score with Uniqueness: This looks at the amount of duplicate values in the Acc_ID column (Under the assumption that the account id should be unique). This is represented by the function *DQ_score_unique()*

2. DQ Score without Uniqueness: This score does not make any assumptions about the uniqueness of the acc_id column. This is represented by the function *DQ_score()*

In [54]:
def DQ_score_unique():
    final_metrices = [accuracy(), duplication(), uniqueness(),
                 completeness(), validity()]
    return sum(final_metrices)/len(final_metrices)

In [55]:
def DQ_score():
    final_metrices = [accuracy(), duplication(),
                 completeness(), validity()]
    return sum(final_metrices)/len(final_metrices)


#### Attaching the result to a DataFrame and exporting the DataFrame for Visualization

In [56]:
#Attach the results to a list
final_metrices = [[DQ_score_unique(), DQ_score(), accuracy(), 
                   duplication(), uniqueness(), completeness(), validity()]]
# Turn it into a DataFrame
Result_df = pd.DataFrame(final_metrices, columns = ['DQ_score_unique', 'DQ_score', 'Accuracy', 'Duplication', 
                                                    'Uniqueness', 'Completeness', 'Validity'])

In [57]:
#Export Data to an excel file
Result_df.to_csv('DQ_metrices.csv')

#### Delete Duplicated rows from the Errors Table & Export the Clean Table and error tables to csv files

In [59]:
Errorsss_df = Errorsss_df.drop_duplicates(keep='last') # Drop duplicates and Keep Only the Last Entry
Errorsss_df.to_csv('DQ_Issues.csv')

This allows for both additional analysis of the errors in any tool of choice but also a pertinent information for data owners to advise on there data quality issues are in order to adequately correct them

In [60]:
Errorsss_df.shape

(738, 12)

In [61]:
Errorsss_df

,acc_id,account_type,credit_turnover,debit_turnover,month_data_date,acc_status,min_balance,balance,rejected_payments,monthly_payment_scheduled,credit_limit,error_code
1574,32211554.0,A,4031.0,3027.0,2021-06-03 00:00:00,D,##############################################...,1913-09-12 00:00:00,0,3,5000,input in min_balance column is not a number
1578,32211554.0,A,956.0,5305.0,2021-08-04 00:00:00,D,##############################################...,1907-06-05 00:00:00,0,3,5000,input in min_balance column is not a number
1580,32211554.0,A,4110.0,6724.0,2021-09-04 00:00:00,D,##############################################...,1900-04-08 00:00:00,0,3,5000,input in min_balance column is not a number
1582,32211554.0,A,4402.0,4607.0,2021-10-05 00:00:00,D,##############################################...,##############################################...,0,3,5000,input in min_balance column is not a number
1585,47123208.0,A,2895.0,3557.0,2021-06-03 00:00:00,D,-12510,3338,0,4,1000,input in min_balance column is an outlier
...,...,...,...,...,...,...,...,...,...,...,...,...
2328,47126976.0,A,2500.0,3113.0,2021-11-05 00:00:00,D,NaN,NaN,0,7,1000,input in balance column is not a number
2329,32211554.0,A,468.0,4871.0,2021-11-05 00:00:00,D,1901-07-08 00:00:00,##############################################...,0,3,5000,input in balance column does not meet length r...
2330,21321547.0,B,2500.0,2814.0,2021-10-05 00:00:00,D,-695,A,A,A,A,input in rejected_payments column is not a number
2331,21321547.0,B,2500.0,2814.0,2021-10-05 00:00:00,D,-695,A,A,A,A,input in monthly_payment_scheduled column is n...


In [62]:
cleaned_df = pd.merge(cleaned_df, Errorsss_df, how='left', indicator=True)
cleaned_df = cleaned_df[cleaned_df['_merge'] == 'left_only'].drop(columns=['_merge'])


In [63]:
cleaned_df.shape

(27019, 12)

In [64]:
cleaned_df.to_csv('DQ_Clean.csv')

## Thank You.